In [1]:
import requests
import json

- Get all shops

In [6]:
url = "https://www.foody.vn/__get/Place/HomeListPlace?CategoryGroup=food&page=1&count=12&dtids=1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,2,18,693,696,699,695,694,698&districtId=&cateId=&cuisineId=&isReputation=&type=1"
all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)
print(dataJson)

{'CityId': 217, 'ExcludeIds': None, 'Query': None, 'Items': [{'Id': 725868, 'Name': 'Ái Vy - Sinh Tố & Nước Ép', 'Address': '37 Công Chúa Ngọc Hân, P. 13, Quận 11, TP. HCM', 'AvgRating': 8.598, 'AvgRatingText': '8.6', 'RestaurantStatus': 2, 'Phone': '0902 341 324', 'PhotoUrl': 'https://images.foody.vn/res/g73/725868/prof/s640x400/foody-upload-api-foody-mobile-83dc5dcd2daac3f49abb-180327114748.jpg', 'TotalReviews': 11, 'TotalFavourites': 0, 'TotalViews': 0, 'TotalPictures': 65, 'TotalCheckIns': 0, 'Url': '/ho-chi-minh/ai-vy-sinh-to-nuoc-ep', 'Tags': None, 'Description': None, 'Note': {'RestaurantId': 0, 'UserId': 0, 'ListId': None, 'ListName': None, 'CreatedOn': '/Date(-62135596800000)/', 'Description': None, 'Id': 0}, 'HasPromotion': False, 'HasMemberCard': False, 'MemberCardDiscount': None, 'Latitude': 10.7632651, 'Longitude': 106.6549186, 'ResUsername': None, 'ResCreatedOn': '/Date(-62135596800000)/', 'LstPicture': [], 'LstReview': [{'reviewProfileUrl': '/thanh-vien/sinhtoaivy', 'rev

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

- get detail of ONE shop

In [40]:
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

{'reply': {'delivery_detail': {'total_order': 0, 'rating': {'total_review': 500, 'avg': 4.7, 'display_total_review': '500+', 'app_link': 'foody://foody.vn?action=reviewlist&id=1031704'}, 'is_subscribe': False, 'is_favorite': False, 'city_id': 217, 'phones': ['0776969606'], 'restaurant_id': 1031704, 'district_id': 16, 'brand_id': 11607, 'video': None, 'asap_is_available': False, 'contract_type': 1, 'id': 121227, 'location_url': 'ho-chi-minh', 'foody_service_id': 1, 'is_quality_merchant': True, 'available_times': [{'date': '2022-09-21', 'times': ['09:55', '10:00', '10:05', '10:10', '10:15', '10:20', '10:25', '10:30', '10:35', '10:40', '10:45', '10:50', '10:55', '11:00', '11:05', '11:10', '11:15', '11:20', '11:25', '11:30', '11:35', '11:40', '11:45', '11:50', '11:55', '12:00', '12:05', '12:10', '12:15', '12:20', '12:25', '12:30', '12:35', '12:40', '12:45', '12:50', '12:55', '13:00', '13:05', '13:10', '13:15', '13:20', '13:25', '13:30', '13:35', '13:40', '13:45', '13:50', '13:55', '14:00',